# Predictive Election NN Model

Using the cleaned registration and election result data, built a neural network model for predicting outcomes of elections. Input features are:

* Partisan registration in upcoming election
* Partisan registration in prior election
* Outcome of prior election

One of the challenges with predicting election outcomes in Colorado is the variability due to about a third of the state being unaffiliated with the two major parties. The behavior of those unaffiliateds with vary by district, so we need a model that can account for that.

We have a pretty limited dataset, using the following combinations

| Inputs | Output |
|---------|----------|
|2012 reg., results, 2016 reg. | 2016 results |
|2014 reg., results, 2018 reg. | 2018 results |
|2016 reg., results, 2020 reg. | 2020 results |

with the final goal of using the 2016 reg. and results + the most current 2020 reg. to predict the 2020 results.

In [ ]:
# Useful imports
import pandas as pd

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GaussianNoise
from tensorflow import keras

from matplotlib import pyplot as plt

import numpy as np

from tqdm.notebook import tqdm

# needed for our loss plots
from IPython.display import clear_output

%matplotlib inline

res_dir = '../data/results/cleaned'
reg_dir = '../data/registration/cleaned'

build_models = False

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()      
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.loglog(self.x, self.losses, label="loss")
        plt.loglog(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.ylabel('error')
        plt.xlabel('epoch')
        plt.show();
        
plot_losses = PlotLosses()

## Data Preparation

Need to arrange the cleaned CSV files into the inputs and outputs with appropriate groupings.

Senate elections are every four years, so prior results from 2012 will be paired with registration in 2016 to predict the 2016 outcome. Similarly with 2014 and 2018.

First, define some helper functions that will pre-process the data from the cleaned registration and results files

In [ ]:
def get_registration(year):
    """Given an input year, snag the registration data for that year and collapse the affiliation of active
    voters into REP, DEM, OTHER
    
    Returns a dataframe with columns DISTRICT-COUNTY, REP, DEM, OTHERS"""
    
    # read in the data
    reg_df = pd.read_csv(reg_dir+'/senate_{}.csv'.format(year))
    
    # filter out where the county value is empty, which is a "total" row
    reg_df = reg_df[reg_df['COUNTY'].notnull()]
    
    # isolate only active voters
    active_cols = [col for col in reg_df.columns if '-ACTIVE' in col]
    
    # find third-party/unaffiliated voter data
    other_cols = [col for col in active_cols if 'DEM' not in col and 'REP' not in col]
    
    # combine all unaffiliateds into one column
    reg_df['OTHER-ACTIVE'] = reg_df[other_cols].sum(axis=1)
    
    # combine the district and county labels into one
    reg_df['DIST_COUNTY'] = reg_df['DISTRICT'].astype(str) +'-'+ reg_df['COUNTY'].astype(str)

    # pick off only the interesting data
    relabel_dict = {
        'REP-ACTIVE' : 'REP',
        'DEM-ACTIVE' : 'DEM',
        'OTHER-ACTIVE' : 'OTHER'
    }
    new_df = reg_df[['DIST_COUNTY', 'REP-ACTIVE', 'DEM-ACTIVE', 'OTHER-ACTIVE']].rename(columns=relabel_dict)
    
    return new_df
    
def get_results(year):
    """Given an input year, snag the results data for that year and collapse the affiliation of active
    candidates into REP, DEM, OTHER
    
    Returns a dataframe with columns DISTRICT-COUNTY, REP, DEM, OTHERS"""

    # read in the results file
    df = pd.read_csv(res_dir+'/senate_{}.csv'.format(year))
    
    # create a DIST-COUNTY label
    df['DIST_COUNTY'] = df['DISTRICT'].astype(str) + '-' + df['COUNTY'].astype(str)
    
    # isolate third party candidates
    parties = ['DEMOCRATIC PARTY', 'REPUBLICAN PARTY']
    df['PARTY'][~df['PARTY'].isin(parties)] = 'OTHER'
    df['PARTY'][df['PARTY'] == 'REPUBLICAN PARTY'] = 'REP'
    df['PARTY'][df['PARTY'] == 'DEMOCRATIC PARTY'] = 'DEM'

    # sum over precincts, if precincts exist
    agg_cols = {'YES VOTES' : 'sum'}
    df = df.groupby([df['DIST_COUNTY'], df['PARTY']], as_index=False).aggregate(agg_cols)
    
    # Make party votes into columns for each DIST_COUNTY
    df = df.pivot(index='DIST_COUNTY', columns='PARTY', values='YES VOTES').fillna(0)
    df.reset_index(level=0, inplace=True)

    return df

At this point, it is helpful to combine the all the relevant input and output data for each dist-county into a single input and output dataframe.

In [ ]:
# Will store the district-county data using the standard key format 
# 'SD [NUMBER]-[COUNTY]' for the Year of the Predicted Results
year_pairs = [['2012', '2016'], ['2014', '2018']]

X_set = np.array([], dtype=np.float64).reshape(0,9)
Y_set = np.array([], dtype=np.float64).reshape(0,3)

dfs = []

for pair in year_pairs:
    
    # The output variables
    current_result = get_results(pair[1])
    elections = current_result.DIST_COUNTY.unique()
    current_result = current_result.rename(
        columns={'DEM' : 'DEM_curr_votes',
                 'REP' : 'REP_curr_votes',
                 'OTHER' : 'OTHER_curr_votes'}
    )
        
    # The input variables
    
    # current registration
    current_reg = get_registration(pair[1])
    current_reg = current_reg[current_reg['DIST_COUNTY'].isin(elections)]
    
    current_reg = current_reg.rename(
        columns={'DEM' : 'DEM_curr_reg',
                 'REP' : 'REP_curr_reg',
                 'OTHER' : 'OTHER_curr_reg'}
    )
    
    # prior election registration
    past_reg = get_registration(pair[0])
    past_reg = past_reg[past_reg['DIST_COUNTY'].isin(elections)]
    
    past_reg = past_reg.rename(
        columns={'DEM' : 'DEM_past_reg',
                 'REP' : 'REP_past_reg',
                 'OTHER' : 'OTHER_past_reg'}
    )
    
    # prior election result registration
    past_result = get_results(pair[0])
    past_result = past_result[past_result['DIST_COUNTY'].isin(elections)]
    
    past_result = past_result.rename(
        columns={'DEM' : 'DEM_past_votes',
                 'REP' : 'REP_past_votes',
                 'OTHER' : 'OTHER_past_votes'}
    )
    
    # compute fractional current registration
    tot_regs = current_reg.sum(axis=1).values
    current_reg.DEM_curr_reg  = current_reg.DEM_curr_reg.values/tot_regs
    current_reg.REP_curr_reg  = current_reg.REP_curr_reg.values/tot_regs
    current_reg.OTHER_curr_reg = current_reg.OTHER_curr_reg.values/tot_regs
    
    # compute fractional past registration
    tot_regs = past_reg.sum(axis=1).values
    past_reg.DEM_past_reg  = past_reg.DEM_past_reg.values/tot_regs
    past_reg.REP_past_reg  = past_reg.REP_past_reg.values/tot_regs
    past_reg.OTHER_past_reg = past_reg.OTHER_past_reg.values/tot_regs
    
    # compute fractional past votes
    tot_votes = past_result.sum(axis=1).values
    past_result.DEM_past_votes = past_result.DEM_past_votes.values/tot_votes
    past_result.REP_past_votes = past_result.REP_past_votes.values/tot_votes
    past_result.OTHER_past_votes = past_result.OTHER_past_votes.values/tot_votes

    
    X_df = current_reg
    X_df = pd.merge(X_df, past_reg, on='DIST_COUNTY')
    X_df = pd.merge(X_df, past_result, on='DIST_COUNTY')
    
    # Now for the outputs
    # compute current fractional votes
    tot_votes = current_result.sum(axis=1).values
    current_result.DEM_curr_votes = current_result.DEM_curr_votes.values/tot_votes
    current_result.REP_curr_votes = current_result.REP_curr_votes.values/tot_votes
    current_result.OTHER_curr_votes = current_result.OTHER_curr_votes.values/tot_votes

    X_df = pd.merge(X_df, current_result, on='DIST_COUNTY')
    
    dfs.append(X_df)

In [ ]:
result = pd.concat(dfs)
X_cols = ['REP_curr_reg', 'DEM_curr_reg', 'OTHER_curr_reg',
          'REP_past_reg', 'DEM_past_reg', 'OTHER_past_reg',
          'REP_past_votes', 'DEM_past_votes', 'OTHER_past_votes']
Y_cols = ['DEM_curr_votes', 'OTHER_curr_votes', 'REP_curr_votes']

print(result[Y_cols])

X_set = result[X_cols].values
Y_set = result[Y_cols].values

## Model Training

### Split the Data

Split into a train-test split to validate the data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
print(np.shape(X_set))
X_test, X_train, Y_test, Y_train = train_test_split(X_set, Y_set, train_size=.2)
print(np.shape(X_test))

### Train a Model

This is a small dataset, so we need to use a very shallow neural network

In [ ]:
model = Sequential()

model.add(Dense(2, input_dim=9, activation='tanh'))
model.add(GaussianNoise(stddev=.1))
#model.add(Dense(2, activation='tanh'))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
hist = model.fit(x=X_train, y=Y_train, 
                    validation_data= (X_test, Y_test),
                    batch_size = 10,
                    shuffle = True,
                    epochs = 500,
                    verbose = False,
                    callbacks = [plot_losses])

In [ ]:
plt.figure()
plt.loglog(hist.epoch, hist.history['val_loss'], label = 'val. loss')
plt.loglog(hist.epoch, hist.history['loss'], label = 'loss')
plt.xlabel('epoch')
plt.ylabel('validation loss')
plt.legend()
plt.tight_layout()
plt.savefig('election_model.png')
pass;

### Train an ensemble of models

Because of the small number of data points, uncertainty quantification is important. So, build an ensemble of models from different training data, and then make the predictions using the average/spread of each model.

For a 20% train-test split, there are $\binom{89}{71} \approx 3 \times 10^{18}$ possible combinations. So, we aren't too worried about repeats.

In [ ]:
if build_models:
    n_models = 1000
    train_size = .3
    models_dict = {}

    losses = []
    vals = []

    for model_num in tqdm(range(n_models)):

        X_test, X_train, Y_test, Y_train = train_test_split(X_set, Y_set, train_size=train_size)
        model = Sequential()

        model.add(Dense(2, input_dim=9, activation='tanh'))
        model.add(GaussianNoise(stddev=.1))
        model.add(Dense(3, activation='sigmoid'))

        model.compile(loss='mean_squared_error', optimizer='adam')

        hist = model.fit(x=X_train, y=Y_train, 
                        validation_data= (X_test, Y_test),
                        batch_size = 20,
                        shuffle = True,
                        epochs = 500,
                        verbose = False)

        models_dict['model {}'.format(model_num)] = model
        losses.append(hist.history['loss'][-1])
        vals.append(hist.history['val_loss'][-1])

    #    print('final loss = {}'.format(hist.history['loss'][-1]))
    #    print('final val. loss = {}\n'.format(hist.history['val_loss'][-1]))
    for model in models_dict:
    
        models_dict[model].save('./model/{}'.format(model))
    
    plt.scatter(losses, vals)
pass;

## 2020 Predictions

In [ ]:
pair = ['2016', '2020']

# The input variables

# prior election result registration
past_result = get_results(pair[0])
elections = past_result['DIST_COUNTY'].unique()
past_result = past_result[past_result['DIST_COUNTY'].isin(elections)]

past_result = past_result.rename(
    columns={'DEM' : 'DEM_past_votes',
             'REP' : 'REP_past_votes',
             'OTHER' : 'OTHER_past_votes'}
)

# current registration
current_reg = get_registration(pair[1])
current_reg = current_reg[current_reg['DIST_COUNTY'].isin(elections)]

current_reg = current_reg.rename(
    columns={'DEM' : 'DEM_curr_reg',
             'REP' : 'REP_curr_reg',
             'OTHER' : 'OTHER_curr_reg'}
)

# prior election registration
past_reg = get_registration(pair[0])
past_reg = past_reg[past_reg['DIST_COUNTY'].isin(elections)]

past_reg = past_reg.rename(
    columns={'DEM' : 'DEM_past_reg',
             'REP' : 'REP_past_reg',
             'OTHER' : 'OTHER_past_reg'}
)

# compute fractional current registration
tot_regs = current_reg.sum(axis=1).values
current_reg.DEM_curr_reg  = current_reg.DEM_curr_reg.values/tot_regs
current_reg.REP_curr_reg  = current_reg.REP_curr_reg.values/tot_regs
current_reg.OTHER_curr_reg = current_reg.OTHER_curr_reg.values/tot_regs

# compute fractional past registration
tot_regs = past_reg.sum(axis=1).values
past_reg.DEM_past_reg  = past_reg.DEM_past_reg.values/tot_regs
past_reg.REP_past_reg  = past_reg.REP_past_reg.values/tot_regs
past_reg.OTHER_past_reg = past_reg.OTHER_past_reg.values/tot_regs

# compute fractional past votes
tot_votes = past_result.sum(axis=1).values
past_result.DEM_past_votes = past_result.DEM_past_votes.values/tot_votes
past_result.REP_past_votes = past_result.REP_past_votes.values/tot_votes
past_result.OTHER_past_votes = past_result.OTHER_past_votes.values/tot_votes


X_df = current_reg
X_df = pd.merge(X_df, past_reg, on='DIST_COUNTY')
X_df = pd.merge(X_df, past_result, on='DIST_COUNTY')

X_pred = X_df[X_cols].values

Iterate over all models, get the predicted vote totals by district-county for each model, and store. We will eventually compute total and spread of votes in each district

In [ ]:
import glob

models = glob.glob('./model/*')

current_reg = get_registration(pair[1])
current_reg = current_reg[current_reg['DIST_COUNTY'].isin(elections)]
tot_regs = current_reg.sum(axis=1).values
tot_regs = np.reshape(tot_regs, (len(tot_regs),1))

preds_array = []

for model in tqdm(models):
    this_model = keras.models.load_model(model)
    preds = this_model.predict(X_pred)
    preds_array.append(preds*tot_regs)

In [ ]:
mean_df = pd.DataFrame(data = {'DIST_COUNTY' : current_reg.DIST_COUNTY.values,
                              'DEM' : np.mean(preds_array, axis=0)[:,0],
                              'REP' : np.mean(preds_array, axis=0)[:,2],
                              'OTHER' : np.mean(preds_array, axis=0)[:,1]})

print(mean_df)

std_df = pd.DataFrame(data = {'DIST_COUNTY' : current_reg.DIST_COUNTY.values,
                              'DEM' : np.std(preds_array, axis=0)[:,0],
                              'REP' : np.std(preds_array, axis=0)[:,2],
                              'OTHER' : np.std(preds_array, axis=0)[:,1]})

print(std_df)

In [ ]:
mean_df.plot.bar(x='DIST_COUNTY')
std_df.plot.bar(x='DIST_COUNTY')
pass;

In [ ]:
mean_df['DISTRICT'] = mean_df['DIST_COUNTY'].str.split('-').str[0]
std_df['DISTRICT'] = std_df['DIST_COUNTY'].str.split('-').str[0]

mean_df['COUNTY'] = mean_df['DIST_COUNTY'].str.split('-').str[1]
std_df['COUNTY'] = std_df['DIST_COUNTY'].str.split('-').str[1]

print(mean_df)
print(std_df)

### Election Predictions

Here are the total vote predictions for each district. We start with a bar chart and STD range of predictions. We will also look at the individual predictions and win percentages.

In [ ]:
import matplotlib as mpl

# Use the Gill Sans font
import matplotlib.pylab as pylab

import matplotlib.font_manager as fm
font = fm.FontProperties(
       family = 'Gill Sans',
       fname = '/usr/share/fonts/truetype/adf/GilliusADF-Regular.otf')
params = {
         'font.size': 14,
         'font.style': 'italic',
         'legend.fontsize': 14,
         'figure.figsize': (9.5, 6),
         'axes.labelsize': 14,
         'axes.titlesize': 16,
         'xtick.labelsize': 14,
         'ytick.labelsize': 14,
         'axes.spines.right' : False,
         'axes.spines.left' : False,
         'axes.spines.top' : False,
         'axes.spines.bottom' : False,
         'font.family' : 'Gill Sans'}
pylab.rcParams.update(params)


width = .25

for district in mean_df.DISTRICT.unique():
    fig, ax = plt.subplots()
    
    ax.set_title('District {} Predictions'.format(district));
    
    counties = mean_df[mean_df.DISTRICT == district].COUNTY.values
    
    dem_votes   = mean_df[mean_df.DISTRICT == district].DEM.values
    rep_votes   = mean_df[mean_df.DISTRICT == district].REP.values
    other_votes = mean_df[mean_df.DISTRICT == district].OTHER.values
    
    dem_std   = std_df[mean_df.DISTRICT == district].DEM.values
    rep_std   = std_df[mean_df.DISTRICT == district].REP.values
    other_std = std_df[mean_df.DISTRICT == district].OTHER.values
    
    counties = np.append(counties, 'TOTAL')
    
    dem_votes = np.append(dem_votes, np.sum(dem_votes))
    dem_std = np.append(dem_std, np.sum(dem_std))
    
    rep_votes = np.append(rep_votes, np.sum(rep_votes))
    rep_std = np.append(rep_std, np.sum(rep_std))
    
    other_votes = np.append(other_votes, np.sum(other_votes))
    other_std = np.append(other_std, np.sum(other_std))
    
    
    x = np.arange(len(counties))
    
    ax.bar(x - width, dem_votes, width, label=counties, yerr=2.*dem_std, color='tab:blue', alpha=0.5);
    ax.bar(x, rep_votes, width, label=counties, yerr = 2.*rep_std, color='tab:red', alpha=0.5);
    ax.bar(x+width, other_votes, width, label=counties, yerr = 2.*other_std, color='tab:green', alpha=0.5);
    
    ax.set_xticks(x)
    for tick in ax.get_xticklabels():
        tick.set_rotation(85)
    ax.set_xticklabels(counties)
    ax.get_yaxis().set_major_formatter(
        mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    ax.annotate(format(int(dem_votes[-1]), ','),
            xy=(x[-1]-width, dem_votes[-1]),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha='center', va='bottom');
    
    ax.annotate(format(int(rep_votes[-1]), ','),
            xy=(x[-1], rep_votes[-1]),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha='center', va='bottom');
    
    ax.annotate(format(int(other_votes[-1]), ','),
            xy=(x[-1]+width, other_votes[-1]),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha='center', va='bottom');
    
    
    fig.tight_layout()
    
    fig.savefig('figs/forecasts/senate/{}_projection.png'.format(district))
    
pass;

### Spread in Projections

Now we want to look at each model's predictions for each district, to figure out whether things are clustered, driven by outliers, etc. Key questions here are what percentage of models predict a Dem versus Rep win?

First, some data cleaning. Pandas doesn't like multidimensional arrays, so I'm using a dictionary instead.

In [ ]:
print(np.shape(preds_array))
preds_array = np.array(preds_array)

preds_dict = {}

dem_votes = preds_array[:,:,0].T
rep_votes = preds_array[:,:,2].T
oth_votes = preds_array[:,:,1].T

dist_county = current_reg.DIST_COUNTY.values

for idx in range(len(dist_county)):
    preds_dict[dist_county[idx]] = preds_array[:,idx,:]

print(preds_dict.keys())


#dems_df = pd.DataFrame(data = {'DIST_COUNTY' : current_reg.DIST_COUNTY.values,
#                              'votes' : [dem_votes]})

In [ ]:
dists = mean_df.DISTRICT.unique()

dist_tots = {}

for dist in dists:
    tot_votes = 0
    for key in preds_dict.keys():
        if dist == key.split('-')[0]:
            tot_votes += preds_dict[key]
    
    dist_tots[dist] = tot_votes
    

In [ ]:
fig, ax = plt.subplots()
for idx, dist in enumerate(dists):
    
    ax.scatter([2*idx-.2]*np.shape(dist_tots[dist])[0], dist_tots[dist][:,0], c='tab:blue', alpha=0.5, s=5)
    ax.scatter([2*idx]*np.shape(dist_tots[dist])[0], dist_tots[dist][:,1],  c='tab:green', alpha=0.5, s=5)
    ax.scatter([2*idx+.2]*np.shape(dist_tots[dist])[0], dist_tots[dist][:,2],  c='tab:red', alpha=0.5, s=5)
    
ax.set_xticks(np.linspace(0,2*len(dists)-2, len(dists)))
ax.set_xticklabels(dists, rotation='vertical', fontsize=18)
ax.set_title('Prediction Ranges')
fig.tight_layout()
fig.savefig('senate_full_predictions.png')
pass;